In [ ]:
%matplotlib inline
from skimage import io
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy import ndimage
from scipy.fft import *
from skimage.filters import window

In [ ]:
def display_image(image, vmin=0, vmax=255):
    plt.figure(figsize=(6, 6))
    plt.imshow(image, cmap='gray', vmin=vmin, vmax=vmax)
    plt.show()
    
def display_fft(im_fft, phase=False, log=True):
    im_magnitude = np.abs(im_fft)
    if log: im_magnitude = np.log10(im_magnitude)
    plt.figure(figsize=(12, 6))
    if phase:
        im_phase = np.angle(im_fft)
        plt.subplot(1, 2, 1)
        plt.imshow(im_magnitude, cmap='gray')
        plt.subplot(1, 2, 2)
        plt.imshow(im_phase, cmap='gray')
    else:
        plt.imshow(im_magnitude)
    plt.show()
        
def display_in3d(image):
    plt.figure(figsize=(6, 6))
    xx, yy = np.mgrid[0:image.shape[0], 0:image.shape[1]]

    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot_surface(xx, yy, image ,rstride=1, cstride=1, cmap=cm.coolwarm, linewidth=0)

    # show it
    plt.show()

In [ ]:
def cree_damier(taille_image, taille_damier):
    rep_d = int(taille_image / (2 * taille_damier))
    return np.kron([[255, 0] * rep_d, [0, 255] * rep_d] * rep_d, np.ones((taille_damier, taille_damier)))

taille_damier, taille_carre = 96, 0.5
zoom_factor = int(np.ceil(1/taille_carre))

print(f'Taille du damier = {taille_damier} | Taille des carrés = {taille_carre} | Facteur d\'agrandissement = {zoom_factor}')
damier_original = cree_damier(96*zoom_factor, int(taille_carre*zoom_factor))
#display_image(damier_original)  # affichage du damier original
damier = ndimage.zoom(damier_original, 1/zoom_factor, order=0)
display_image(damier)  # affichage du damier

In [ ]:
image = io.imread('cerveau.png')
im_fft = fftshift(fft2(image))
display_fft(im_fft, phase=True)
display_in3d(np.log10(np.abs(im_fft)))

In [ ]:
def create_square(taille_image=256, taille_carre=None, offset_centre=None):
    """
    Cette fonction crée un carré dans une image de 256x256, avec les dimensions passées en paramètres
    """
    if taille_carre is None:
        taille_carre = (50, 50)
    if offset_centre is None:
        offset_centre = (0, 0)
        
    arr = np.zeros((taille_image, taille_image))
    slicex = slice(int(round(offset_centre[0]+taille_image/2-taille_carre[0]/2)), int(round(offset_centre[0]+taille_image/2+taille_carre[0]/2)))
    slixey = slice(int(round(offset_centre[1]+taille_image/2-taille_carre[1]/2)), int(round(offset_centre[1]+taille_image/2+taille_carre[1]/2)))
    arr[slicex, slixey] = 255
    return arr.astype(np.uint8)

In [ ]:
# Valeurs
# taille_carre=(50, 50) (50, 5) (5, 50)
# offset_centre=(0, 0) (10, 0)

im_carre = create_square(taille_image=256, taille_carre=(5, 50), offset_centre=(0, 0))
plt.imshow(im_carre)
im_fft = fftshift(fft2(im_carre))
display_fft(im_fft, phase=False, log=False)

In [ ]:
image = io.imread('cerveau.png')
im_fft = fftshift(fft2(image))

# Manipulation de la fft avant reconstruction inverse (ifft)

im_fft_mod = im_fft
#im_fft_mod = np.real(im_fft)
#im_fft_mod = np.imag(im_fft)
#im_fft_mod = np.abs(im_fft)
#im_fft_mod = np.angle(im_fft)

display_fft(im_fft_mod, phase=True)
image_recon = ifft2(ifftshift(im_fft_mod))
display_image(np.real(image_recon))  # attention, l'image est complexe

In [ ]:
# Exemple de code de la transformée de fourier d'une image * gaussienne 2D

damier = cree_damier(250, 125)
display_image(damier)
damier_fft = fftshift(fft2(damier))

display_fft(damier_fft, phase=False, log=False)
wdamier_fft = damier_fft * window(('gaussian', 2), damier_fft.shape)
display_fft(wdamier_fft, phase=False, log=False)

damier_recon = ifft2(ifftshift(wdamier_fft))
display_image(np.real(damier_recon))  # attention, l'image est complexe

In [ ]:
# Exemple de zero-padding

damier = cree_damier(250, 125)
damier_pad = np.pad(damier, 250)
display_image(damier_pad)

damier_fft = fftshift(fft2(damier_pad))

display_fft(damier_fft, phase=False, log=False)
wdamier_fft = damier_fft * window(('gaussian', 6), damier_fft.shape)
display_fft(wdamier_fft, phase=False, log=False)

damier_recon = ifft2(ifftshift(wdamier_fft))

damier_extract = damier_recon[250:-250, 250:-250]
display_image(np.real(damier_extract))  # attention, l'image est complexe


In [ ]:
# Que se passe-t-il quand on pad la fft?
image = io.imread('cerveau.png')
#image = cree_damier(250, 125)
display_image(image)

image_fft = fftshift(fft2(image))
image_fft_pad = np.pad(image_fft, image.shape[0])

#display_fft(image_fft_pad, phase=False, log=False)

image_recon = ifft2(ifftshift(image_fft_pad))

print(image_recon.shape)
display_image(np.real(image_recon), vmax=None)  # attention, l'image est complexe

In [ ]:
from skimage.draw import disk

# Filtre idéal
image = io.imread('cerveau.png')

image_filtre = np.zeros((image.shape[0], image.shape[1]), dtype=np.double)
rr, cc = disk((image.shape[0]/2, image.shape[1]/2), image.shape[0]*0.25, shape=image.shape)
image_filtre[rr, cc] = 1

image_fft = fftshift(fft2(image))
image_fft_mod = image_fft * image_filtre

display_fft(image_fft, phase=False)
display_fft(image_fft_mod, phase=False)
display_in3d(image_filtre)

image_recon = ifft2(ifftshift(image_fft_mod))
display_image(np.real(image_recon), vmax=None)

In [ ]:
# Fenetrage
from skimage.filters import window

# Filtre idéal avec fenêtrage
image = io.imread('cerveau.png')

image_filtre = np.zeros((image.shape[0], image.shape[1]), dtype=np.double)
rr, cc = disk((image.shape[0]/2, image.shape[1]/2), image.shape[0]*0.25, shape=image.shape)
image_filtre[rr, cc] = 1
image_filtre_tmp = ifftshift(ifft2(image_filtre))
image_filtre_tmp = image_filtre_tmp * window(('gaussian', 6), image_filtre_tmp.shape)
image_filtrew = fft2(image_filtre_tmp)

image_fft = fftshift(fft2(image))
image_fft_mod = image_fft * image_filtrew

display_fft(image_fft, phase=False)
display_fft(image_fft_mod, phase=False)
display_in3d(np.abs(image_filtrew))

image_recon = ifftshift(ifft2(image_fft_mod))
display_image(np.abs(image_recon), vmax=None)



In [ ]:
# Filtre Gaussien et de Butterworth
image = io.imread('cerveau.png')

def create_Butter2D(shape, freqn, n):
    centre = shape[0]/2, shape[1]/2
    freq = freqn * (shape[0] + shape[1]) / 4
    filtre_butter = np.zeros((shape[0], shape[1]), dtype=np.double)
    for u in range(shape[0]):
        for v in range(shape[1]):
            distance = np.sqrt((u-centre[0])**2 + (v-centre[1])**2)
            filtre_butter[u, v] = 1 / (1 + (distance / freq) ** (2*n))
    return filtre_butter

def create_Gaussian2D(shape, freqn):
    centre = shape[0]/2, shape[1]/2
    freq = freqn * (shape[0] + shape[1]) / 4
    filtre_gaussian = np.zeros((shape[0], shape[1]), dtype=np.double)
    for u in range(shape[0]):
        for v in range(shape[1]):
            distance = np.sqrt((u-centre[0])**2 + (v-centre[1])**2)
            filtre_gaussian[u, v] = np.exp(-distance ** 2 / (2 * freq ** 2))
    return filtre_gaussian

image_fft = fftshift(fft2(image))
#filtre_PB = create_Butter2D(image_fft.shape, freqn=0.25, n=8)
filtre_PB = create_Gaussian2D(image_fft.shape, freqn=0.25)

# Passe-bas
#image_fft_mod = image_fft * filtre_PB
#display_in3d(np.abs(filtre))

# Passe-haut
filtre_HP = (1 - filtre_PB)
#image_fft_mod = image_fft * filtre_HP
#display_in3d(np.abs(filtre_HP))

# Rehaussement
K = 2
filtre_HB = 1 + K * filtre_HP
image_fft_mod = image_fft * filtre_HB
display_in3d(np.abs(filtre_HB))


image_recon = ifft2(ifftshift(image_fft_mod))
display_image(np.abs(image_recon), vmax=None)

In [ ]:
# Filtre homomorphique

def create_Homomorphique(shape, lambdaLow, lambdaHigh, c, freqn):
    centre = shape[0]/2, shape[1]/2
    freq = freqn * (shape[0] + shape[1]) / 4
    filtre_homomorphique = np.zeros((shape[0], shape[1]), dtype=np.double)
    for u in range(shape[0]):
        for v in range(shape[1]):
            distance = np.sqrt((u-centre[0])**2 + (v-centre[1])**2)
            filtre_homomorphique[u, v] = (lambdaHigh - lambdaLow) * (1 - np.exp(-c * distance ** 2 / (freq ** 2))) + lambdaLow
    return filtre_homomorphique

filtre = create_Homomorphique(image_fft.shape, lambdaLow=0.75, lambdaHigh=2, c=1, freqn=0.25)
display_in3d(np.abs(filtre))

image = io.imread('cerveau.png')
image_ln = np.log10(image+0.01)
image_fft = fftshift(fft2(image_ln))
image_fft_mod = image_fft * filtre
image_recon = ifft2(fftshift(image_fft_mod))
image_exp = np.exp(image_recon)

print(np.min(image), np.max(image))
print(np.min(image_exp), np.max(image_exp))

display_image(image, vmin=0, vmax=255)
display_image(np.abs(image_exp), vmin=0, vmax=15)